<a href="https://colab.research.google.com/github/sftSalman/tensoflowMaster/blob/main/VGG_implementaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds 

In [9]:
class Myclass:
  def __init__(self):
    self.v1 = 1

my_obj = Myclass()

In [11]:
my_obj.__dict__

{'v1': 1}

In [12]:
vars(my_obj)

{'v1': 1}

In [13]:
# Add a new instance variable and give it a value
my_obj.var2 = 2

# Calls vars() again to see the object's instance variables
vars(my_obj)

{'v1': 1, 'var2': 2}

In [14]:
# Call vars, passing in the object.  Then access the __dict__ dictionary using square brackets
vars(my_obj)['var3'] = 3

# Call vars() to see the object's instance variables
vars(my_obj)

{'v1': 1, 'var2': 2, 'var3': 3}

In [26]:
for i in range(4,10):
    # Format a string that is var
    vars(my_obj)[f'var{i}'] = 0
    
# View the object's instance variables!
vars(my_obj)

{'var1': 1, 'var4': 0, 'var5': 0, 'var6': 0, 'var7': 0, 'var8': 0, 'var9': 0}

In [18]:
type(vars)

builtin_function_or_method

In [22]:
i=1
print(f"var{i}")

var1


In [23]:
class MyClass:
    def __init__(self):
        # Use vars(self) to access the class's dictionary of variables
        vars(self)['var1'] = 1

# Create an object of type MyClass()
my_obj = MyClass()
vars(my_obj)

{'var1': 1}

In [31]:
class Block(tf.keras.Model):
  def __init__(self,filters,kernel_size,repetations,pool_size=2,strides = 2):
    super().__init__()
    self.filters = filters
    self.kernel_size = kernel_size
    self.repetations = repetations


    for i in range(self.repetations):
      vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(self.filters,
                                                         self.kernel_size,
                                                         activation = 'relu',
                                                         padding='same')
    self.max_pool = tf.keras.layers.MaxPool2D(pool_size,strides=strides)


  def call(self,inputs):
    conv2D_0 = vars(self)['conv2D_0']

    x = conv2D_0(inputs)

    for i in range(1, self.repetations):
      conv2D_i = vars(self)[f'conv2D_{i}']

      x = conv2D_i(x)
    max_pool = self.max_pool(x)

    return max_pool

In [35]:
class MyVGG(tf.keras.Model):
  def __init__(self, num_classe):
    super().__init__()

    self.block_a = Block(filters=64,kernel_size=3,repetations=2)
    self.block_b = Block(128,3,2)
    self.block_c = Block(256,3,3)
    self.block_d = Block(512,3,3)
    self.block_e = Block(512,3,3)
    

    self.flatten = tf.keras.layers.Flatten()
    self.fc = tf.keras.layers.Dense(258,activation='relu')
    self.classifier = tf.keras.layers.Dense(num_classe,activation='softmax')



  def call (self, inputs):

    x = self.block_a(inputs)
    x = self.block_b(x)
    x = self.block_c(x)
    x = self.block_d(x)
    x = self.block_e(x)
    x = self.flatten(x)
    x = self.fc(x)
    x = self.classifier(x)
    return x


In [36]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

# Initialize VGG with the number of classes 
vgg = MyVGG(num_classe=2)

# Compile with losses and metrics
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define preprocessing function
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., features['label']

# Apply transformations to dataset
dataset = dataset.map(preprocess).batch(32)

# Train the custom VGG model
vgg.fit(dataset, epochs=10)


Epoch 1/10
  1/727 [..............................] - ETA: 3:28:17 - loss: 0.6931 - accuracy: 0.5312

KeyboardInterrupt: ignored